# 17. Calculating compilers

## 17.1 Introduction

## 17.2 Syntax and semantics

In [3]:
data Expr = Val Int | Add Expr Expr

eval :: Expr -> Int
eval (Val n)   = n
eval (Add x y) = eval x + eval y 

In [5]:
eval (Add (Val 1) (Val 2))

3

## 17.3 Adding a stack

In [8]:
type Stack = [Int]

eval' :: Expr -> Stack -> Stack
eval' (Val n)   = push n
eval' (Add x y) = add . eval' y . eval' x

push :: Int -> Stack -> Stack
push n s = n : s

add :: Stack -> Stack
add (m : n : s) = m+n : s

```haskell
eval' e s == eval e : s
```
Redefining `eval` in terms of `eval'` using the property above.

In [17]:
eval :: Expr -> Int
eval e = head (eval' e [])

## 17.4 Adding a continuation

In [13]:
type Cont = Stack -> Stack

eval'' :: Expr -> Cont -> Cont
eval'' (Val n)   c = c . push n
eval'' (Add x y) c = eval'' x . eval'' y $ c . add

```haskell
eval'' e c s == c (eval' e s) 
```

Redefining `eval'` in terms of `eval''` using the property above.

In [18]:
eval' :: Expr -> Cont
eval' e = eval'' e id

## 17.5 Defunctionalising

In [34]:
haltC :: Cont
haltC = id

pushC :: Int -> Cont -> Cont
pushC n k = k . push n

addC :: Cont -> Cont
addC k = k . add

In [35]:
eval' :: Expr -> Cont
eval' e = eval'' e haltC

eval'' :: Expr -> Cont -> Cont
eval'' (Val n)   = pushC n
eval'' (Add x y) = eval'' x . eval'' y . addC

In [36]:
data Code = HALT | PUSH Int Code | ADD Code
          deriving Show

In [37]:
:type HALT
:type PUSH
:type ADD

HALT :: Code

PUSH :: Int -> Code -> Code

ADD :: Code -> Code

In [39]:
exec :: Code -> Cont
exec HALT       = haltC
exec (PUSH n c) = pushC n (exec c)
exec (ADD c)    = addC (exec c)

In [1]:
exec :: Code -> Stack -> Stack
exec HALT       s           = s
exec (PUSH n c) s           = exec c (n : s)
exec (ADD c)    (m : n : s) = exec c (n+m : s)

: 

In [41]:
comp' (Val n)   = PUSH n
comp' (Add x y) = comp' x . comp' y . ADD

## 17.6 Combining the steps

## 17.7 Chapter remarks

## 17.8 Exercises